# 梯度与优化器学习记录笔记

**配合教案使用，不再全部记录，主要记录每一个板块产生的疑问和精华**

## 一、梯度及其产生的问题

**梯度的定义：**  
梯度是多元函数在某一点处变化率最大的方向向量，它指示了函数值增长最快的方向。在深度学习中，
梯度通常指损失函数相对于每个模型参数的偏导数。

**梯度更新的基本原理：**  
梯度下降是优化算法的核⼼思想，通过沿着梯度的反方向更新参数来最小化损失函数。

**梯度下降的三种变体：**  
1、批量梯度下降。  
2、随机批量梯度下降。（SGD）  
3、小批量梯度下降。

**梯度在LLM训练中的重要性：**  
1、参数更新：梯度指导参数如何减少来调整预测错误。  
2、学习信号：模型从数据中获取反馈的机制。当模型预测错误时，计算出的梯度就包含了“哪里错了”以及“应该朝哪个方向修正”的信息。  
3、特征学习：通过反向传播，梯度帮助模型学习有用的更复杂的特征表示。这是LLM能够理解语言、生成连贯文本的基础。

**梯度消失：**  
在深层网络中，梯度在反向传播过程中逐渐变小，最终接近于0，导致网络浅层参数几乎不更新。  
原因一般是：  
1、激活函数饱和：如sigmoid函数在输入过大过小的时候接近于0.  
2、网络层数过深：梯度连乘导致数值越来越小。  
3、权重初始化不当：初始权重过小。

影响一般是：  
1、前层网络学习缓慢或停滞。  
2、模型无法学习到复杂特征表示。  
3、训练效果不佳

**梯度爆炸：**  
在深层网络中，梯度在方向传播过程中逐渐变大，最终导致数值溢出和参数更新过大。   
原因一般是：  
1、权重初始化过大  
2、学习率设置过高  
3、网络结构设计问题

影响一般是：  
1、参数更新过大，模型不稳定。  
2、损失函数值剧烈波动。  
3、训练过程发散。

**上述两种情况的解决方案：**  
1、选用优秀的权重初始化方法。  
2、选用合适的激活函数。   
3、使用批归一化。  
4、使用残差连接机制。  
5、使用梯度裁剪方法。

**梯度检查点技术：（Gradient Checkpointing）**  
“激活值”的概念：本质上就是每一层经过非线性变换后的输出。  
梯度检查点概念：一种内存优化技术，在标准反向传播中，必须保存所有激活值来计算梯度，但检查点技术则是通过在前向传播过程中只保存部分关键位置的中间激活值（即“检查点”），而在反向传播过程中，当需要某个未保存的激活值时，从最近的检查点重新执行前向计算来恢复该值。这种方法以增加计算量为代价，显著减少了训练过程中的内存占用，实现了内存与计算开销之间的权衡。  


工作流程：  
1、前向传播：计算并保存检查点层的激活值丢弃非检查点层的激活值。   
2、反向传播：从最近的检查点开始，重新计算到目标层的激活值。使用重新计算的激活值计算梯度。

作用：  
1、节省内存，减少GPU显存占用。  
2、支持更大的模型。  
3、权衡计算和内存，以增加计算时间的代价节省内存

**梯度累积技术：（Gradient Accumulation）**  
概念：梯度累积是一种模拟大批量训练的技术，通过多次小批量计算梯度并累积，最后一次性更新参数。    
原理：在标准的小批量训练中，每次前向传播和反向传播后立即更新参数。而在梯度累积中，多次小批量的梯度被累积起来，等累积到指定次数后再更新参数。  

更新公式：  
$$
\theta_{t+1}=\theta_t-\frac{\mu}{n}\sum_{i=1}^n \nabla_{\theta}J_i(\theta_t)
$$  
其中n是累计步数，$J_i$是第i个小批量的损失。

解决的问题：  
1、显存不足：当GPU显存无法支持大批量训练时，通过梯度累积模拟大批量效果。  
2、训练稳定性：大批量训练通常更稳定，梯度累积可以实现类似效果。  
3、泛化性能：大批量训练有助于提高模型的泛化能力。  

**梯度裁剪技术：（Gradient Clipping）**  
梯度裁剪是⼀种防止**梯度爆炸的技术**，通过限制梯度的范数来稳定训练过程。当梯度的范数超过设定阈值时，按比例缩小梯度，使其不超过阈值。

法一：按范数裁剪  
核心思想：  
将整个模型的所有梯度看作一个巨大的向量，计算这个向量的“总长度”（即范数），如果这个“总长度”超过了预设的阈值（threshold），就按比例缩小整个梯度向量，使其“长度”刚好等于阈值。   
优点：  
保持梯度方向：裁剪后，梯度向量的方向不变，只是长度被缩短。这保留了梯度提供的“学习信号”的相对关系。  
全局控制：从整体上控制了梯度的“能量”，非常稳定  
计算步骤：  
1、将模型中所有可训练参数的梯度拼接成一个长向量g，然后计算其L2范数，$||g||_2$表示向量g的L2范数。
$$
{||g||}_2=\sqrt{\sum_i g_i^2}
$$  
其中$g_i$是第i个参数的梯度

2、比较并裁剪.  
如果$||g||$<=threshold，那就不进行操作。  
如果$||g||$>threshold，则对所有梯度进行等比例缩放。
$$
g=g \cdot \frac{threshold}{||g||_2}
$$

法二：按值裁剪
核心思想：对每一个单独的梯度值进行限制。如果某个梯度的值超过了预设的上限，就把它设为上限值；如果低于预设的下限，就把它设为下限值。  
优点：  
简单直观：实现容易，逻辑清晰。  
局部控制：可以防止个别梯度值过大。  
缺点：  
破坏梯度方向：它粗暴地截断了极端值，可能会改变梯度向量的整体方向，丢失部分学习信号。  
不够智能：即使只有一个梯度非常大，其他梯度正常，它也只修改那个大的值，而按范数裁剪会整体缩放，更均衡。  

## 二、优化器及其相关问题

**优化器的作用：**  
优化器是深度学习训练中的核⼼组件，负责根据梯度信息更新模型参数。其主要作用包括：  
1、参数更新：根据梯度信息调整模型参数。  
2、学习率调整：动态调整学习率以提高训练效果。  
3、动量控制：利用历史梯度信息加速收敛。    
4、自适应调整：根据不同参数的特点进行个性化调整。  

**常见优化器的类型：**  
1、⼀阶优化器：基于梯度信息进行参数更新。    
2、⼆阶优化器：基于梯度和⼆阶导数信息进行参数更新。    
3、自适应优化器：自动调整学习率和其他超参数。  

**SGD优化器：（随机梯度下降）**  
适用场景：适用于对泛化性能要求较高的任务，或计算资源有限的场景。  
基本公式：  
$\theta_{t+1}=\theta_t-\eta\nabla_{\theta}J(\theta_t)$  
优点：  
1、简单高效，泛化能力好。
2、理论基础扎实，数学收敛性分析成熟。
3、适合大规模数据。  
缺点：  
1、收敛速度慢，训练时间长。  
2、对学习率敏感：学习率太大 → 损失震荡，甚至发散。学习率太小 → 收敛太慢，陷入局部极小值。   
3、路径震荡严重（无动量时）  
4、容易陷入局部极小值。  

**Momentum优化器：**  
基本公式：  
$v_{t+1}=\gamma v_t+\eta\nabla_{\theta}J(\theta_t)$  
$\theta_{t+1}=\theta_t-v_{t+1}$  
其中$v_t$是动量项，$\gamma$是动量系数，通常取0.9

原理：引入动量项，积累历史梯度信息，使得参数更新具有惯性，能够加速收敛并减少震荡  
作用：  
1、加速收敛：在梯度方向一致的时候加速更新。  
2、减少震荡：在梯度方向变化的时候减少震荡。  
3、跳出局部最优：借助惯性有可能跳出局部最优。

**RMSProp优化器：**  
基本公式：  
$s_{t+1}=\gamma s_t+(1-\gamma)[\nabla_{\theta}J(\theta_t)]^2$  
$\theta_{t+1}=\theta_t-\frac{\eta}{\sqrt{s_{t+1}+\epsilon}}\nabla_{\theta}J(\theta_t)$  
其中,$s_t$是梯度平方的指数移动平均，$\epsilon$是平滑项（防止除零）

原理：自适应调整学习率，对不同参数使用不同的学习率。历史梯度大的参数使用较小的学习率，历史梯度小的参数使
用较大的学习率。  
作用：  
1、自适应学习率：根据历史梯度信息自动调整学习率  
2、稳定训练：减少训练过程中的波动  
3、快速收敛：在非凸优化问题中表现良好  

**Adam优化器：（结合了RMSProp和Momentum的优点）**  
适用场景：适用于大多数深度学习任务，特别是需要快速收敛的场景。  
基本公式：  
$m_{t+1}=\beta_1 m_t+(1-\beta_1)\nabla_{\theta}J(\theta_t) $  
$s_{t+1}=\beta_2 s_t +(1-\beta_2)[\nabla_{\theta}J(\theta_t)]^2$  
$\hat{m_{t+1}}=\frac{m_{t+1}}{1-\beta_1^t}$     
$\hat{s_{t+1}}=\frac{s_{t+1}}{1-\beta_2^t}$  
$$
\theta_{t+1}=\theta_t-\frac{\eta}{\sqrt{\hat{s_{t+1}}+\epsilon}}\hat{m_{t+1}}
$$
其中，$m_t$是梯度的⼀阶矩估计，$s_t$是梯度的⼆阶矩估计

原理：考虑了梯度的历史信息，又对不同参数使用自适应学习率。  
作用：  
1、动量加速：利用历史梯度信息加速收敛   
2、自适应学习率：对不同参数使用不同的学习率
3、偏差修正：通过偏差修正项提高初期估计的准确性  
常用参数：  
$\beta_1=0.9$（一阶矩估计的衰减率）  
$\beta_2=0.999$（二阶矩估计的衰减率）  
$\epsilon=10^{-8}$（平滑项）  

**AdamW优化器：**  
在Adam的基础上改进了权重衰减的处理方式。  
基本公式：  
$m_{t+1}=\beta_1 m_t+(1-\beta_1)\nabla_{\theta}J(\theta_t) $  
$s_{t+1}=\beta_2 s_t +(1-\beta_2)[\nabla_{\theta}J(\theta_t)]^2$  
$\hat{m_{t+1}}=\frac{m_{t+1}}{1-\beta_1^t}$     
$\hat{s_{t+1}}=\frac{s_{t+1}}{1-\beta_2^t}$  
$$
\theta_{t+1}=\theta_t-\eta(\frac{\hat{m_{t+1}}}{\sqrt{\hat{s_{t+1}}+\epsilon}}+\lambda \theta_t)
$$  
其中，$\lambda$是权重衰减系数。

优化点：将权重衰减与梯度更新分离，直接在参数更新时加入权重衰减项，而不是在损失函数中加入L2正则化项。在参数更新时同时考虑了梯度信息和权重衰减，有助于提高模型的泛化性能。  
优势：  
1、更准确的权重衰减：避免了Adam中权重衰减与L2正则化等价性的误解。  
2、更好的泛化性能：在许多任务上表现优于Adam。  
3、更稳定的训练：减少过拟合风险。  

**Muon优化器：**  
概念：Muon是⼀种专门为大预言模型设计的优化器，结合了Adam和SGD的优点，在训练大模型时表现出色。  
核心思想：  
1、对重要部分参数使用Adam优化器来加速收敛。  
2、对其他不那么重要参数使用SGD优化器来减少内存占用。  
3、通过智能分配策略决定哪些参数使用哪种优化器。

优势：    
1、高效训练：在大模型训练中比纯Adam或纯SGD更高效。    
2、内存友好：相比Adam减少内存占用。   
3、收敛稳定：结合了两种优化器的优点。  
4、计算效率：减少了需要存储的优化器状态数量，降低了计算开销。  

缺点：  
Muon有训练崩溃问题。  
1、梯度方差大：对于某些参数组，梯度的方差可能非常大，导致参数更新剧烈波动。  
2、自适应机制失效：在某些情况下，Muon的自适应机制可能无法有效控制梯度更新。  
3、混合优化的不协调：不同优化策略之间的切换可能导致训练过程不稳定。

**Loss缩放和学习率调整上的区别：**  
结论：对于SGD这种基础单阶优化器，没有区别。但对于Adam等高阶优化器就有区别。  
为什么会有区别？  
见教案，提示：可以从推导公式上求解。  
在Adam优化器中，Loss缩放和学习率调整会产生完全不同的优化路径，这是因为Loss缩放改变了梯度的幅度，进而影响了自适应学习率的计算，而学习率调整只改变了最终的更新步长而不影响自适应学习率的计算过程。

**Kimi-K2的Muon-clip：**  
提出背景：    
1、稳定性需求：在大规模模型训练中，稳定性比速度更重要。  
2、实际应用挑战：在实际训练中发现，纯Muon优化器在某些阶段会出现梯度爆炸现象。  
3、优化策略改进：需要在保持Muon优势的同时，增强训练的鲁棒性。  

原理：  
结合了Muon优化器和梯度裁剪的思想。  
1、参数分组：  
将模型参数分为不同的组，对不同组的参数采用不同的优化策略。  
2、自适应裁剪：  
根据参数的重要性动态调整梯度裁剪阈值，对重要参数使用较小的裁剪阈值，对非重要参数使用较大的裁剪阈值。  
3、混合优化：  
对关键参数使用Adam优化器以保证收敛速度对其他参数使用SGD优化器以提高泛化性能。  

Kimi解决崩溃问题的方：  
1、梯度范数控制：  
实时监控梯度范数，防止梯度爆炸。对异常大的梯度进行裁剪，保持更新的稳定性。  
2、动态阈值调整：  
根据训练阶段动态调整裁剪阈值。在训练初期使用较宽松的阈值，后期使用较严格的阈值。  
3、分层保护机制：  
对不同层的参数采用不同的保护策略。关键层使用更严格的保护措施。

优势：  
1、高效训练：结合了不同优化器的优点。  
2、稳定收敛：通过自适应裁剪保证训练稳定性。  
3、资源优化：针对不同参数采用最适合的优化策略。  